In [1]:
import pandas as pd

C:\Users\ral\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import json
from datetime import datetime

In [45]:
df = pd.read_json('nba_betting/nba_betting_data.json')

In [46]:
df = df.dropna(how="any")

In [47]:
df = df.drop_duplicates(keep='first')

In [48]:
df[(df.season=="2024-25") & (df.team=="Boston Celtics")]

,team,season,date,matchup,final_score,spread_outcome,closing_spread,total_outcome,closing_total
0,Boston Celtics,2024-25,Feb 26,@ DET,L 97-117,L,-4.5,U,224.0
1,Boston Celtics,2024-25,Feb 25,@ TOR,W 111-101,L,-11.0,U,226.5
2,Boston Celtics,2024-25,Feb 23,NY,W 118-105,W,-7.5,U,231.0
3,Boston Celtics,2024-25,Feb 20,@ PHI,W 124-104,W,-6.5,O,225.5
4,Boston Celtics,2024-25,Feb 12,SA,W 116-103,W,-8.0,U,229.5
...,...,...,...,...,...,...,...,...,...
61,Boston Celtics,2024-25,Oct 12,PHI,W 139-89,W,-7.5,O,222.5
62,Boston Celtics,2024-25,Oct 6,DEN,W 130-104,W,-2.5,O,216.5
63,Boston Celtics,2024-25,Oct 4,@ DEN,W 107-103,W,-1.0,U,217.5
82784,Boston Celtics,2024-25,Mar 2,DEN,W 110-103,W,-5.5,U,236.5


In [49]:
df[(df.season=="2022-23") & (df.team=="Brooklyn Nets")].shape

(90, 9)

In [50]:
df = df.rename(columns={'team': 'TEAM_NAME', 'date': 'GAME_DATE', 'matchup': 'MATCHUP', 'season': 'SEASON_YEAR'})

In [51]:
print(df.shape)
df.head()

(15124, 9)


,TEAM_NAME,SEASON_YEAR,GAME_DATE,MATCHUP,final_score,spread_outcome,closing_spread,total_outcome,closing_total
0,Boston Celtics,2024-25,Feb 26,@ DET,L 97-117,L,-4.5,U,224.0
1,Boston Celtics,2024-25,Feb 25,@ TOR,W 111-101,L,-11.0,U,226.5
2,Boston Celtics,2024-25,Feb 23,NY,W 118-105,W,-7.5,U,231.0
3,Boston Celtics,2024-25,Feb 20,@ PHI,W 124-104,W,-6.5,O,225.5
4,Boston Celtics,2024-25,Feb 12,SA,W 116-103,W,-8.0,U,229.5


In [52]:
df['TEAM_NAME'] = df['TEAM_NAME'].replace({'LA Clippers': 'Los Angeles Clippers'})

In [53]:
def convert_date(row):
    season = row['SEASON_YEAR']
    date = row['GAME_DATE']
    if season == '2024-25':
        if 'Feb' in date or 'Jan' in date or 'Mar' in date:
            return date + ' 2025'
        else:
            return date + ' 2024'
        
    elif season == '2023-24':
        if 'Feb' in date or 'Jan' in date or 'Mar' in date or 'Apr' in date or 'May' in date or 'Jun' in date:
            return date + ' 2024'
        else:
            return date + ' 2023'

    elif season == '2022-23':
        if 'Feb' in date or 'Jan' in date or 'Mar' in date or 'Apr' in date or 'May' in date or 'Jun' in date:
            return date + ' 2023'
        else:
            return date + ' 2022'
    
    elif season == '2021-22':
        if 'Feb' in date or 'Jan' in date or 'Mar' in date or 'Apr' in date or 'May' in date or 'Jun' in date:
            return date + ' 2022'
        else:
            return date + ' 2021'
        
    elif season == '2020-21':
        if 'Feb' in date or 'Jan' in date or 'Mar' in date or 'Apr' in date or 'May' in date or 'Jun' in date:
            return date + ' 2021'
        else:
            return date + ' 2020'
    
    elif season == '2019-20':
        if 'Feb' in date or 'Jan' in date or 'Mar' in date or 'Apr' in date or 'May' in date or 'Jun' in date:
            return date + ' 2020'
        else:
            return date + ' 2019'

In [54]:
df['GAME_DATE'] = df.apply(convert_date, axis=1)

In [55]:
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'], format='%b %d %Y')

In [56]:
order = ['SEASON_YEAR', 'GAME_DATE', 'TEAM_NAME', 'MATCHUP']
other = [col for col in df.columns if col not in order]
df = df[order + other]
df.head()

,SEASON_YEAR,GAME_DATE,TEAM_NAME,MATCHUP,final_score,spread_outcome,closing_spread,total_outcome,closing_total
0,2024-25,2025-02-26,Boston Celtics,@ DET,L 97-117,L,-4.5,U,224.0
1,2024-25,2025-02-25,Boston Celtics,@ TOR,W 111-101,L,-11.0,U,226.5
2,2024-25,2025-02-23,Boston Celtics,NY,W 118-105,W,-7.5,U,231.0
3,2024-25,2025-02-20,Boston Celtics,@ PHI,W 124-104,W,-6.5,O,225.5
4,2024-25,2025-02-12,Boston Celtics,SA,W 116-103,W,-8.0,U,229.5


In [57]:
def home_teams(row):
    matchup = row['MATCHUP']
    if matchup[0] == '@':
        return matchup[2:]
    else:
        return row['TEAM_NAME']

In [58]:
df['H_TEAM_NAME'] = df.apply(home_teams, axis=1)

In [59]:
def away_teams(row):
    matchup = row['MATCHUP']
    if matchup[0] == '@':
        return row['TEAM_NAME']
    else:
        return matchup

In [60]:
df['A_TEAM_NAME'] = df.apply(away_teams, axis=1)

In [61]:
nba_team_abbreviation_mapping = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "Los Angeles Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP",
    "New York Knicks": "NYK",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Washington Wizards": "WAS"
}


In [62]:
df["H_TEAM_NAME"] = df["H_TEAM_NAME"].replace(nba_team_abbreviation_mapping)
df["A_TEAM_NAME"] = df["A_TEAM_NAME"].replace(nba_team_abbreviation_mapping)
df

,SEASON_YEAR,GAME_DATE,TEAM_NAME,MATCHUP,final_score,spread_outcome,closing_spread,total_outcome,closing_total,H_TEAM_NAME,A_TEAM_NAME
0,2024-25,2025-02-26,Boston Celtics,@ DET,L 97-117,L,-4.5,U,224.0,DET,BOS
1,2024-25,2025-02-25,Boston Celtics,@ TOR,W 111-101,L,-11.0,U,226.5,TOR,BOS
2,2024-25,2025-02-23,Boston Celtics,NY,W 118-105,W,-7.5,U,231.0,BOS,NY
3,2024-25,2025-02-20,Boston Celtics,@ PHI,W 124-104,W,-6.5,O,225.5,PHI,BOS
4,2024-25,2025-02-12,Boston Celtics,SA,W 116-103,W,-8.0,U,229.5,BOS,SA
...,...,...,...,...,...,...,...,...,...,...,...
156991,2024-25,2025-03-01,Philadelphia 76ers,GS,W 126-119,W,6.5,O,228.0,PHI,GS
160134,2024-25,2025-03-02,New York Knicks,@ MIA,W 116-112,L,-6.0,O,219.5,MIA,NYK
160190,2024-25,2025-02-28,New York Knicks,@ MEM,W 114-113,W,2.0,U,244.0,MEM,NYK
162243,2024-25,2025-03-01,Brooklyn Nets,@ DET,L 94-115,L,12.5,U,216.5,DET,BKN


In [63]:
df.A_TEAM_NAME.unique()

array(['BOS', 'NY', 'SA', 'DAL', 'CHI', 'HOU', 'ATL', 'ORL', 'NO', 'SAC',
       'TOR', 'IND', 'PHI', 'DET', 'MEM', 'MIL', 'MIA', 'LAC', 'MIN',
       'CLE', 'BK', 'GS', 'DEN', 'BKN', 'UTA', 'CHA', 'PHO', 'WAS', 'LAL',
       'OKC', 'POR', 'SAS', 'NOP', 'PHX', 'GSW', 'NYK'], dtype=object)

In [64]:
fixed_map = {
    'NY': 'NYK',
    'NO': 'NOP',
    'PHO': 'PHX',
    'SA': 'SAS',
    'GS': 'GSW',
    'BK': 'BKN'
}

In [65]:
df["H_TEAM_NAME"] = df["H_TEAM_NAME"].replace(fixed_map)
df["A_TEAM_NAME"] = df["A_TEAM_NAME"].replace(fixed_map)
df

,SEASON_YEAR,GAME_DATE,TEAM_NAME,MATCHUP,final_score,spread_outcome,closing_spread,total_outcome,closing_total,H_TEAM_NAME,A_TEAM_NAME
0,2024-25,2025-02-26,Boston Celtics,@ DET,L 97-117,L,-4.5,U,224.0,DET,BOS
1,2024-25,2025-02-25,Boston Celtics,@ TOR,W 111-101,L,-11.0,U,226.5,TOR,BOS
2,2024-25,2025-02-23,Boston Celtics,NY,W 118-105,W,-7.5,U,231.0,BOS,NYK
3,2024-25,2025-02-20,Boston Celtics,@ PHI,W 124-104,W,-6.5,O,225.5,PHI,BOS
4,2024-25,2025-02-12,Boston Celtics,SA,W 116-103,W,-8.0,U,229.5,BOS,SAS
...,...,...,...,...,...,...,...,...,...,...,...
156991,2024-25,2025-03-01,Philadelphia 76ers,GS,W 126-119,W,6.5,O,228.0,PHI,GSW
160134,2024-25,2025-03-02,New York Knicks,@ MIA,W 116-112,L,-6.0,O,219.5,MIA,NYK
160190,2024-25,2025-02-28,New York Knicks,@ MEM,W 114-113,W,2.0,U,244.0,MEM,NYK
162243,2024-25,2025-03-01,Brooklyn Nets,@ DET,L 94-115,L,12.5,U,216.5,DET,BKN


In [66]:
df = df.drop(columns=['TEAM_NAME', 'MATCHUP'])

In [67]:
df = df.rename(columns={"H_TEAM_NAME": "H_TEAM_ABBREVIATION", "A_TEAM_NAME": "A_TEAM_ABBREVIATION"})

In [68]:
order = ['SEASON_YEAR', 'GAME_DATE', 'H_TEAM_ABBREVIATION', 'A_TEAM_ABBREVIATION']
other = [col for col in df.columns if col not in order]
df = df[order + other]
df = df.drop_duplicates(subset=['GAME_DATE', 'H_TEAM_ABBREVIATION', 'A_TEAM_ABBREVIATION'], keep='first')
df.head()

,SEASON_YEAR,GAME_DATE,H_TEAM_ABBREVIATION,A_TEAM_ABBREVIATION,final_score,spread_outcome,closing_spread,total_outcome,closing_total
0,2024-25,2025-02-26,DET,BOS,L 97-117,L,-4.5,U,224.0
1,2024-25,2025-02-25,TOR,BOS,W 111-101,L,-11.0,U,226.5
2,2024-25,2025-02-23,BOS,NYK,W 118-105,W,-7.5,U,231.0
3,2024-25,2025-02-20,PHI,BOS,W 124-104,W,-6.5,O,225.5
4,2024-25,2025-02-12,BOS,SAS,W 116-103,W,-8.0,U,229.5


In [69]:
df = df.drop(columns=['final_score', 'SEASON_YEAR'])

In [70]:
FINAL = pd.read_csv('FINAL.csv', low_memory=False)
FINAL = FINAL.drop(columns='Unnamed: 0')
print(FINAL.shape)

(6733, 284)


In [71]:
def truncate_date(row):
    date = row['GAME_DATE']
    return date[:10]

In [72]:
FINAL["GAME_DATE"] = FINAL.apply(truncate_date, axis=1)

In [73]:
FINAL["GAME_DATE"] = pd.to_datetime(FINAL["GAME_DATE"])

In [74]:
FINAL['H_TEAM_NAME'] = FINAL['H_TEAM_NAME'].replace({'LA Clippers': 'Los Angeles Clippers'})
FINAL['A_TEAM_NAME'] = FINAL['A_TEAM_NAME'].replace({'LA Clippers': 'Los Angeles Clippers'})

In [75]:
FINAL.H_TEAM_NAME.nunique()

30

In [76]:
df.H_TEAM_ABBREVIATION.nunique()

30

In [77]:
merged_df = pd.merge(
    FINAL,
    df.drop(columns=['total_outcome', 'spread_outcome']),
    on=["GAME_DATE", "H_TEAM_ABBREVIATION", "A_TEAM_ABBREVIATION"],
    how="left"
)

merged_df.shape

(6733, 286)

In [78]:
merged_df.H_TEAM_ABBREVIATION.nunique()

30

In [79]:
test = merged_df.dropna(subset=['SPREAD'])
test.shape

(6733, 286)

In [80]:
test.H_TEAM_ABBREVIATION.nunique()

30

In [81]:
test['closing_spread'] = test['closing_spread'] * -1

In [82]:
test.to_csv('2-27-Data.csv')